# 新闻主题分类数据:

## 读入数据集

In [1]:
import torch
from torchtext.datasets import AG_NEWS
path = r'E:\Notebook\自然语言处理\文本处理的基本的方法\AG_NEWS.data'

In [2]:
import pandas as pd

In [3]:
data = pd.read_csv('./data/ag_news_csv/train.csv')

In [4]:
data

,3,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again."
0,3,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...
1,3,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\ab...
2,3,Iraq Halts Oil Exports from Main Southern Pipe...,Reuters - Authorities have halted oil export\f...
3,3,"Oil prices soar to all-time record, posing new...","AFP - Tearaway world oil prices, toppling reco..."
4,3,"Stocks End Up, But Near Year Lows (Reuters)",Reuters - Stocks ended slightly higher on Frid...
...,...,...,...
119994,1,Pakistan's Musharraf Says Won't Quit as Army C...,KARACHI (Reuters) - Pakistani President Perve...
119995,2,Renteria signing a top-shelf deal,Red Sox general manager Theo Epstein acknowled...
119996,2,Saban not going to Dolphins yet,The Miami Dolphins will put their courtship of...
119997,2,Today's NFL games,PITTSBURGH at NY GIANTS Time: 1:30 p.m. Line: ...


In [5]:
train_iter = AG_NEWS(root=path, split='train')

In [6]:
next(train_iter)

(3,
 "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.")

## 构建词汇表

In [7]:
# from torchtext.data.utils import get_tokenizer
# from torchtext.vocab import build_vocab_from_iterator

# tokenizer = get_tokenizer('basic_english')
# train_iter = AG_NEWS(root=path, split='train')

# def yield_tokens(data_iter):
#     for _, text in data_iter:
#         yield tokenizer(text)

# vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=['<unk>'])
# vocab.set_default_index(vocab['<unk>'])

In [8]:
from torchtext.data.utils import get_tokenizer
from collections import Counter
from torchtext.vocab import vocab
 
tokenizer = get_tokenizer('basic_english')
train_iter = AG_NEWS(root= path, split='train')
counter = Counter()
for (label, line) in train_iter:
    counter.update(tokenizer(line))#统计每次词出现的频率
vocab = vocab(counter, min_freq=1)#构建词汇表，min_freq表示指定最小的频次是1
#print(vocab.freqs) #查看词汇及其对应的频率

In [9]:
# text_pipeline = lambda x: vocab(tokenizer(x))
# label_pipeline = lambda x: int(x) - 1

In [10]:
text_pipeline = lambda x: [vocab[token] for token in tokenizer(x)]
label_pipeline = lambda x: int(x) - 1
#print(text_pipeline('here is the an example'))#文本管道，将字符串转换为索引，
#输出：[475, 21, 2, 30, 5286]


## 设置Dataloader

In [11]:
from torch.utils.data import DataLoader

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [12]:
def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]#offsets：batch中每个文本的长度，如[10, 20, 30, 40]
    for (_label, _text) in batch:
        label_list.append(label_pipeline(_label))
        processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
        text_list.append(processed_text)
        offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0) #设置每个text起始索引下标
    text_list = torch.cat(text_list)
    return text_list.to(device), label_list.to(device), offsets.to(device)

## 定义模型

In [13]:
import torch.nn as nn

In [14]:
class TextClassficationModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextClassficationModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()
    
    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()
        
    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

## 训练、评估、测试

In [15]:
import time

In [16]:
def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 500
    start_time = time.time()
    
    for idx, (label, text, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        predited_label = model(text, offsets)
        loss = criterion(predited_label, label)
        loss.backward()
        #将p范数进行归一化操作，归一到0.1，默认为2范数，从而使得parameters的参数的2范数和为0.1
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        
        total_acc += (predited_label.argmax(1) == label).sum().item()
        total_count +=label.size(0)
        if idx % log_interval == 0 and idx>0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader),
                                              total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()

In [17]:
def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0
    
    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predited_label = model(text, offsets)
            loss = criterion(predited_label, label)
            total_acc += (predited_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count

## 主函数

In [18]:
train_iter = AG_NEWS(root=path, split='train')
num_class = len(set([label for (label, text) in train_iter]))
vocab_size = len(vocab)
emsize = 64

In [19]:
model = TextClassficationModel(vocab_size, emsize, num_class)
model.to(device)

TextClassficationModel(
  (embedding): EmbeddingBag(95810, 64, mode=mean)
  (fc): Linear(in_features=64, out_features=4, bias=True)
)

In [20]:
EPOCHS = 10
LR = 0.1
BATCH_SIZE = 4

In [21]:
critersion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 20, gamma=0.1)

In [22]:
total_accu = None
train_iter, test_iter = AG_NEWS(root=path, split=('train','test'))
train_dataset = list(train_iter)
test_dataset = list(test_iter)
num_train = int(len(train_dataset)*0.95)
split_train_, split_valid_ = torch.utils.data.random_split(train_dataset,[num_train, len(train_dataset)-num_train])

In [23]:
train_dataloader = DataLoader(split_train_, batch_size=BATCH_SIZE,shuffle=True, collate_fn=collate_batch)
valid_dataloader = DataLoader(split_valid_, batch_size=BATCH_SIZE,shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE,shuffle=True, collate_fn=collate_batch)

## 训练模块   

In [24]:
for epoch in range(EPOCHS):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val = evaluate(valid_dataloader)
    if total_accu is not None and total_accu > accu_val:
        scheduler.step()
    else:
        total_accu = accu_val
    print('| end of epoch {:3d} | time: {:5.2f}s | '
              'valid accuracy {:8.3f} '.format(epoch,
                                               time.time() - epoch_start_time,
                                               accu_val))

RuntimeError: CUDA error: CUBLAS_STATUS_NOT_INITIALIZED when calling `cublasCreate(handle)`